In [1]:
%pip install pandas

import pandas as pd
data= pd.read_csv('reviews.csv')


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pandas in c:\users\adey9\onedrive\desktop\sentiment-analysis\venv\lib\site-packages (2.2.3)



In [2]:
data.head()

,rating,date,review,platform
0,2,30 December 2024,"I was using it for long time, but have to stop...",blinkit
1,1,4 November 2024,Loving the fast deliveries and mostly they are...,blinkit
2,1,31 October 2024,The customer support is very disappointing. I ...,blinkit
3,5,29 August 2024,"I've been using Blinkit for a while now, and i...",blinkit
4,2,31 December 2024,Blinkit was my go to app and it was rare that ...,blinkit


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4620 entries, 0 to 4619
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   rating    4620 non-null   int64 
 1   date      4620 non-null   object
 2   review    4620 non-null   object
 3   platform  4620 non-null   object
dtypes: int64(1), object(3)
memory usage: 144.5+ KB


In [4]:
# prompt: I want to Create Sentiment Labels:
# Assign labels based on the rating column:
# 1-2: Negative (Label = 0)
# 3: Neutral (Label = 1)
# 4-5: Positive (Label = 2)
# Preprocess Text:
# Clean text (remove special characters, convert to lowercase, etc.).
# Tokenize and pad sequences if you're not using pre-trained models.

import re

def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    return text

# Create sentiment labels
data['label'] = data['rating'].apply(lambda x: 0 if x <= 2 else (1 if x == 3 else 2))

# Preprocess the 'Review' column
data['review'] = data['review'].astype(str) # ensure review column is string type
data['processed_review'] = data['review'].apply(preprocess_text)

data.head()

,rating,date,review,platform,label,processed_review
0,2,30 December 2024,"I was using it for long time, but have to stop...",blinkit,0,i was using it for long time but have to stop ...
1,1,4 November 2024,Loving the fast deliveries and mostly they are...,blinkit,0,loving the fast deliveries and mostly they are...
2,1,31 October 2024,The customer support is very disappointing. I ...,blinkit,0,the customer support is very disappointing i r...
3,5,29 August 2024,"I've been using Blinkit for a while now, and i...",blinkit,2,ive been using blinkit for a while now and it ...
4,2,31 December 2024,Blinkit was my go to app and it was rare that ...,blinkit,0,blinkit was my go to app and it was rare that ...


In [5]:
%pip install transformers datasets torch scikit-learn


ERROR: Could not find a version that satisfies the requirement torch (from versions: none)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for torch


In [6]:
import os
os.environ["WANDB_DISABLED"] = "true"  # Disable W&B logging

# import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Ensure transformers and datasets are installed
%pip install transformers datasets

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Load the dataset
# df = pd.read_csv("your_dataset.csv")

df = data



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


c:\Users\adey9\OneDrive\Desktop\sentiment-analysis\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [7]:

# Map ratings to sentiment labels
def assign_sentiment(rating):
    if rating in [1, 2]:
        return 0  # Negative
    elif rating == 3:
        return 1  # Neutral
    else:
        return 2  # Positive

df['sentiment'] = df['rating'].apply(assign_sentiment)



In [8]:

%pip install torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Dataset

# Split into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['review'], df['sentiment'], test_size=0.2, random_state=42
)

# Tokenization using a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=512)

train_encodings = tokenize_function(list(train_texts))
test_encodings = tokenize_function(list(test_texts))

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_dict({'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask'], 'labels': train_labels})
test_dataset = Dataset.from_dict({'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask'], 'labels': test_labels})

# Load a pre-trained DistilBERT model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)


ERROR: Could not find a version that satisfies the requirement torch (from versions: none)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for torch


Note: you may need to restart the kernel to use updated packages.


ImportError: 
AutoModelForSequenceClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [ ]:

# Define evaluation metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Training arguments
training_args = TrainingArguments(
    report_to=None,  # Ensure W&B is not used
    output_dir="./results",
    eval_strategy="epoch",  # Updated to "eval_strategy"
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Trainer API
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)


In [ ]:

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()


In [ ]:


import torch

# Test the model on custom inputs
# text = "I love using Blinkit! The deliveries are so fast."
text = "Although the delivery speed was impressive, the customer support failed to address my concerns, leaving me frustrated and disappointed. However, the product quality exceeded expectations, making it a bittersweet experience overall."
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

# Move inputs to the same device as the model
inputs = inputs.to(model.device) # Move input tensors to the GPU

outputs = model(**inputs)
predicted_class = outputs.logits.argmax(-1).item()
sentiment_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
print(f"Predicted Sentiment: {sentiment_map[predicted_class]}")